# 📓 DummyApp

The files in this folder code a custom app that resembles an LLM app with
seperate classes for common app components. None of these components make
network calls but pretend to otherwise operate like real components.

__WARNING__: This example must be run from the git repository as it imports
files that are not included in a PIP distribution.

In [1]:
from pathlib import Path
import sys

repo_path = Path().cwd().parent.parent.parent.resolve()

# If using trulens from the repository, add the parent directory to the path:
sys.path.append(str(repo_path))

In [2]:
from examples.dev.dummy_app.app import DummyApp
from examples.dev.dummy_app.agent import DummyAgent

from trulens_eval import Tru
from trulens_eval.tru_custom_app import TruCustomApp

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
# Create custom app:
ca = DummyApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [4]:
# Show instrumented components and methods.

ta.print_instrumented()

Components:
	TruCustomApp (Other) at 0x348ed5f40 with path __app__
	DummyApp (Custom) at 0x348f89b80 with path __app__.app
	DummyAgent (Custom) at 0x348f8a330 with path __app__.app.agents[0]
	DummyApp (Custom) at 0x348f89fa0 with path __app__.app.agents[0].app
	DummyLLM (Custom) at 0x348f8a120 with path __app__.app.agents[0].app.llm
	DummyMemory (Custom) at 0x348f8a090 with path __app__.app.agents[0].app.memory
	DummyReranker (Custom) at 0x348f8a300 with path __app__.app.agents[0].app.reranker
	DummyRetriever (Custom) at 0x348f8a0f0 with path __app__.app.agents[0].app.retriever
	DummyTemplate (Custom) at 0x348f8a1b0 with path __app__.app.agents[0].app.template
	DummyStackTool (Custom) at 0x348f8a180 with path __app__.app.agents[0].app.tools[0]
	DummyTool (Custom) at 0x348f8a1e0 with path __app__.app.agents[0].app.tools[1]
	DummyTool (Custom) at 0x348f8a2a0 with path __app__.app.agents[0].app.tools[2]
	DummyAgent (Custom) at 0x348f8a690 with path __app__.app.agents[1]
	DummyApp (Custom)

In [5]:
# Normal usage (without tracing):
ca.respond_to_query(query="What is the capital of Indonesia?")

# Instrumented usage:
with ta as recorder:
    response = ca.respond_to_query(query="What is the capital of Indonesia?")

record = recorder.get()

In [6]:
# Show the app output:

# response

In [7]:
# Show the instrumentation record.

# record.model_dump()

In [8]:
# Start the dasshboard. If you running from github repo, include the _dev arg below:
tru.start_dashboard(
    _dev=repo_path,
    force=True
)

Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.17.98.166:61571 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

# DummyApp options

The custom app used above has some configuration options or call variants with
differing behaviours:

- Number of agents and number of tools is configurable via `DummyApp` class
  params/attributes `num_agents` and `num_tools`.

- Simulated delay and memory usage controlled by `Dummy` class `delay` and
  `alloc` attributes. All other components subtype `Dummy` hence also implement
  simulated delays and allocations.

- Non-determinism control via the `Dummy` class `seed` attribute. Randomness in
  some components can be somewhat controlled by this attribute. The control is
  presently incomplete as threading and async usage (described below) is not
  controlled by `seed`.

- Async methods. These begin with the letter "a" followed by the sync method
  name.

- Streaming methods. The main method to app and the generation methods to the
  llm component each have streaming versions (`stream_respond_to_query` and
  `stream`). These produce iterables over strings instead of single strings. The
  async variants add a prefix "a" to these and produce asynchronous iterables.

- Use of parallelism for processing retrieved chunks. This uses either threads
  or async depending on which method variant is called. This is controlled by
  the `DummyApp` class `use_parallel` flag.

- Nested use of custom app inside a custom app component. This is controlled by
  the `DummyAgent` class `use_app` flag.

- Use of trulens_eval to record the invocation of the above nested model. This
  is controlled by the `DummyAgent` class `use_recorder` flag.



In [9]:
# This cell executes the app in the various configuration options describe
# above. The final app output should be identical in all cases.

for use_parallel in [False, True]:
    for use_app in [False, True]:
        for use_recorder in [False]:# , True]: # currently not working with nested recorder
            app_id = f"custom_app (use_parralel={use_parallel} use_app={use_app} use_recorder={use_recorder})"
            print(app_id)

            # Create custom app:
            ca = DummyApp(
                use_parallel=use_parallel,
                comp_kwargs={
                    DummyAgent: {"use_app": use_app, "use_recorder": use_recorder}
                },
            )

            # Create trulens wrapper:
            ta = TruCustomApp(
                ca,
                app_id=app_id,
            )

            # Sync usage
            print("  sync")
            print("    non-streaming")
            print("      untraced\t", ca.respond_to_query(query="What is the capital of Indonesia?"))
            with ta as recorder:
                print("      traced\t", ca.respond_to_query(query="What is the capital of Indonesia?"))

            # Sync streaming usage 
            print("    streaming")
            print("      untraced\t ", end="")
            for chunk in ca.stream_respond_to_query(query="What is the capital of Indonesia?"):
                print(chunk, end="")
            print()
            with ta as recorder:
                print("      traced\t ", end="")
                for chunk in ca.stream_respond_to_query(query="What is the capital of Indonesia?"):
                    print(chunk, end="")
                print()

            # TOFIX: Async usage
            #res1 = await ca.arespond_to_query(query="What is the capital of Indonesia?")
            #with ta as recorder:
            #    resp2 = await ca.arespond_to_query(query="What is the capital of Indonesia?")

            # TOFIX: Async streaming usage...


custom_app (use_parralel=False use_app=False use_recorder=False)
  sync
    non-streaming
      untraced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5.
      traced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5.
    streaming
      untraced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5. 
      traced	 
custom_app (use_parralel=False use_app=True use_recorder=False)
  sync
    non-streaming
      untraced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5.
      traced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5.
    streaming
      untraced	 This is my response to a prompt of length 195 with a model derp with temperature 0.5. 
      traced	 
custom_app (use_parralel=True use_app=False use_recorder=False)
  sync
    non-streaming
      untraced	 This is my response to a prompt of length 195 wi